In [14]:
pip install requests


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [2]:
from elasticsearch import Elasticsearch
es_client = Elasticsearch("http://localhost:9200")
es_client.info()

ConnectionError: Connection error caused by: ConnectionError(Connection error caused by: NewConnectionError(<urllib3.connection.HTTPConnection object at 0x71d4f41b5010>: Failed to establish a new connection: [Errno 111] Connection refused))

In [17]:
from elasticsearch import Elasticsearch
es_client = Elasticsearch("http://localhost:9200")


# Create index name
index_name = "course-zoomcamp"

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-zoomcamp'})

In [18]:
from tqdm.auto import tqdm
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

In [19]:
query = "How do execute a command on a Kubernetes pod?"

In [20]:
search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                }
            }
        }
    }

response = es_client.search(index=index_name, body=search_query)

In [21]:
response

ObjectApiResponse({'took': 254, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 739, 'relation': 'eq'}, 'max_score': 44.50556, 'hits': [{'_index': 'course-zoomcamp', '_id': 'ah4ZXJcBK4e2wyrMBFGs', '_score': 44.50556, '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)', 'section': '5. Deploying Machine Learning Models', 'question': 'How do I debug a docker container?', 'course': 'machine-learning-zoomcamp'}}, {'_index': 'course-zoomcamp', '_id': '-R4ZXJcBK4e2wyrMBlH7', '_score': 35.433445, '_source': {'text': 'Deploy and Access the Kubernetes Dashboard\nLuke', 'section': '10. Kubernetes and TensorFlow Serving', 'question': 'Kuberne

In [22]:
query =  "How do copy a file to a Docker container?"

In [23]:
search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }

response = es_client.search(index=index_name, body=search_query)

result_docs = []
    
for hit in response["hits"]["hits"]:
    result_docs.append(hit['_source'])

result_docs

[{'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I debug a docker container?',
  'course': 'machine-learning-zoomcamp'},
 {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani",
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I copy files from my local machine to docker container?',
 

In [24]:
query = "How do I execute a command in a running docker container?"

In [27]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']} \n\nquestion: {doc['question']} \n\ntext: {doc['text']}"
        
    prompt = prompt_template.format(question=query, context=context).strip()
    print(context)
    print(prompt)
    print(len(prompt))
    return prompt

In [28]:
build_prompt(query, result_docs)

section: 5. Deploying Machine Learning Models 

question: How do I debug a docker container? 

text: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)section: 5. Deploying Machine Learning Models 

question: How do I copy files from my local machine to docker container? 

text: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:/path/in/container
Hrithik Kumar Advanisection: 5. Deploying Machine Learning Models 

question: How do I copy files from a di

'You\'re a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.\nUse only the facts from the CONTEXT when answering the QUESTION.\n\nQUESTION: How do I execute a command in a running docker container?\n\nCONTEXT:\nsection: 5. Deploying Machine Learning Models \n\nquestion: How do I debug a docker container? \n\ntext: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)section: 5. Deploying Machine Learning Models \n\nquestion: How do I copy files from my local machine to docker container? \n\ntext: You can copy files from your local machine into a Docker container using the docker cp command. Here\'s how to do it:\nTo copy a file or directory from your local machine into a running Do

In [29]:
from textwrap import dedent

docs = [
    {
        'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\n'
                'docker run -it --entrypoint bash <image>\n'
                'If the container is already running, execute a command in the specific container:\n'
                'docker ps (find the container-id)\n'
                'docker exec -it <container-id> bash\n'
                '(Marcos MJD)',
        'question': 'How do I debug a docker container?',
    },
    {
        'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\n"
                "To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\n"
                "docker cp /path/to/local/file_or_directory container_id:/path/in/container\n"
                "Hrithik Kumar Advani",
        'question': 'How do I copy files from my local machine to docker container?',
    },
    {
        'text': 'You can copy files from your local machine into a Docker container using the docker cp command. Here\'s how to do it:\n'
                'In the Dockerfile, you can provide the folder containing the files that you want to copy over. The basic syntax is as follows:\n'
                'COPY ["src/predict.py", "models/xgb_model.bin", "./"]\t\t\t\t\t\t\t\t\t\t\tGopakumar Gopinathan',
        'question': 'How do I copy files from a different folder into docker container’s working directory?',
    }
]

context_template = "Q: {question} A: {text}"

context = "\n\n".join([
    context_template.format(question=doc["question"], text=doc["text"])
    for doc in docs
])

prompt_template = dedent("""
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. Use only the facts from the CONTEXT when answering the QUESTION.
QUESTION: {question}
CONTEXT: {context}
""").strip()

query = "How do I execute a command in a running docker container?"

prompt = prompt_template.format(question=query, context=context)

print("Prompt length:", len(prompt))

Prompt length: 1460


In [23]:
import tiktoken
encoding = tiktoken.encoding_for_model("gpt-4o")
tokenlist = encoding.encode(prompt)
len(tokenlist)

323